<a href="https://colab.research.google.com/github/Fouvle/Final_Project_Group_22/blob/main/project_Word2VEC_Predictionipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
pip install gensim


In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import csv
import re
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModel
import tensorflow as tf
import torch
import torchvision
import torchaudio
from gensim.models import Word2Vec
from gensim.models import FastText
import re





### Feature Engineering

In [4]:


# Preprocess the text
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    # Remove punctuation and extra spaces
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip().lower()


In [5]:
# Load the datasets
test_text = pd.read_csv('/content/drive/MyDrive/Project/test.txt', delimiter='\t', encoding='utf-8', quoting=csv.QUOTE_NONE, on_bad_lines='skip')
validation_text = pd.read_csv('/content/drive/MyDrive/Project/dev.txt', delimiter='\t', encoding='utf-8', quoting=csv.QUOTE_NONE, on_bad_lines='skip')
train_text = pd.read_csv('/content/drive/MyDrive/Project/train.txt', delimiter='\t', encoding='utf-8', quoting=csv.QUOTE_NONE, on_bad_lines='skip')


In [6]:
# Apply preprocessing and tokenization to all datasets
train_tokenized = train_text.iloc[:, 0].apply(lambda x: preprocess_text(x).split()).tolist()
validation_tokenized = validation_text.iloc[:, 0].apply(lambda x: preprocess_text(x).split()).tolist()
test_tokenized = test_text.iloc[:, 0].apply(lambda x: preprocess_text(x).split()).tolist()


#### WORD2VEC MODEL

In [7]:
# Initialize and train the Word2Vec model
model = Word2Vec(sentences=train_tokenized, vector_size=100, window=5, min_count=1, workers=4)

# Save the model
model.save("word2vec_model.model")


In [8]:

# Load the trained model
model = Word2Vec.load("word2vec_model.model")


In [9]:
def predict_next_word(model, context, top_n=5):
    # Preprocess the context
    context = preprocess_text(context)
    words = context.split()

    # Use the last word(s) as the context
    if not words:
        return []

    last_word = words[-1]

    if last_word not in model.wv:
        return []

    # Find the top_n most similar words
    similar_words = model.wv.most_similar(last_word, topn=top_n)

    # Extract the words from the similar words list
    next_words = [word for word, similarity in similar_words]

    return next_words


In [10]:
context = "me pɛ"

# Predict the next word
next_words = predict_next_word(model, context)
print("Next word predictions:", next_words)

Next word predictions: ['me', 'bere', 'nni', 'ɔmanpanin', 'faa']


In [11]:

def save_word_embeddings_as_text(model, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for word in model.wv.index_to_key:
            vector = model.wv[word]
            vector_str = ' '.join(map(str, vector))
            file.write(f"{word} {vector_str}\n")

model.wv.save("word_embeddings.kv")



In [12]:
# Evaluate the model
def evaluate_word2vec_model(model, tokenized_sentences, top_n=5):
    total = 0
    correct = 0

    for sentence in tokenized_sentences:
        for i in range(1, len(sentence)):
            context = ' '.join(sentence[:i])
            true_next_word = sentence[i]
            predicted_next_words = predict_next_word(model, context, top_n=top_n)

            if true_next_word in predicted_next_words:
                correct += 1
            total += 1

    accuracy = correct / total
    return accuracy

In [13]:
 # Evaluate on the validation set
accuracy = evaluate_word2vec_model(model, validation_tokenized, top_n=5)
print(f"Validation Accuracy: {accuracy}")

Validation Accuracy: 0.0011996700907250506


In [14]:
# Optimize the Word2Vec model
def optimize_word2vec(sentences, vector_size=200, window=10, min_count=1, workers=4, sg=0, hs=4,negative=0):
    model = Word2Vec(sentences=sentences, vector_size=vector_size, window=window, min_count=min_count, workers=workers, sg=sg, hs=hs, negative = negative)
    return model

In [15]:
# Train the optimized model
optimized_model = optimize_word2vec(train_tokenized)
optimized_model.save("optimized_word2vec_model.model")

In [16]:
# Load the optimized model
optimized_model = Word2Vec.load("optimized_word2vec_model.model")


In [17]:
def predict_next_word(model, context, top_n=5):
    # Preprocess the context
    context = preprocess_text(context)
    words = context.split()

    # Use the last word(s) as the context
    if not words:
        return []

    last_word = words[-1]

    if last_word not in model.wv:
        return []

    # Find the top_n most similar words
    similar_words = model.wv.most_similar(last_word, topn=top_n)

    # Extract the words from the similar words list
    next_words = [word for word, similarity in similar_words]

    return next_words


In [18]:
# Predict next words for the test dataset
def predict_words_for_test_dataset(model, tokenized_sentences, top_n=5):
    predictions = []

    for sentence in tokenized_sentences:
        sentence_predictions = []
        for i in range(1, len(sentence)):
            context = ' '.join(sentence[:i])
            next_words = predict_next_word(model, context, top_n=top_n)
            sentence_predictions.append((context, next_words))
        predictions.append(sentence_predictions)

    return predictions

In [19]:
# Get predictions for the test dataset
test_predictions = predict_words_for_test_dataset(optimized_model, test_tokenized, top_n=5)



In [20]:
# Print predictions
for sentence_predictions in test_predictions:
    for context, next_words in sentence_predictions:
        print(f"Context: '{context}' -> Predicted next words: {next_words}")



Streaming output truncated to the last 5000 lines.
Context: 'ne' -> Predicted next words: ['wɔ', 'nson', 'aduasa', 'tɔ', 'a']
Context: 'baabi' -> Predicted next words: ['bi', 'ɛwɔ', 'wɔde', 'ho', 'ɔde']
Context: 'amamfrafoɔ' -> Predicted next words: ['koraa', 'si', 'bio', 'yɛn', 'nti']
Context: 'wɔ' -> Predicted next words: ['ne', 'a', 'nson', 'aduasa', 'ɛdi']
Context: 'paa' -> Predicted next words: ['sɛ', 'na', 'mu', 'deɛ', 'wɔn']
Context: 'boa' -> Predicted next words: ['ɛnam', 'nanso', 'saa', 'reyɛ', 'aba']
Context: 'no' -> Predicted next words: ['sɛ', 'mu', 'na', 'saa', 'bi']
Context: 'ma' -> Predicted next words: ['no', 'ho', 'sɛ', 'mu', 'bi']
Context: 'onhu' -> Predicted next words: ['aguadi', 'odaso', 'nyinsɛn', 'adum', 'abɛhia']
Context: 'sɛ' -> Predicted next words: ['no', 'mu', 'yi', 'wɔn', 'saa']
Context: 'ɔyɛ' -> Predicted next words: ['sɛ', 'na', 'wɔde', 'yi', 'hɔ']
Context: 'fie' -> Predicted next words: ['kyerɛ', 'mu', 'no', 'biara', 'ma']
Context: 'nnwuma' -> Predicted 

In [21]:
# Save word embeddings as text
def save_word_embeddings_as_text(model, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for word in model.wv.index_to_key:
            vector = model.wv[word]
            vector_str = ' '.join(map(str, vector))
            file.write(f"{word} {vector_str}\n")

save_word_embeddings_as_text(optimized_model, "optimized_word_embeddings.txt")

In [22]:
# Evaluate the optimized model on the test dataset
test_accuracy = evaluate_word2vec_model(optimized_model, test_tokenized, top_n=5)
print(f"Test Accuracy: {test_accuracy}")

Test Accuracy: 3.8488184127472866e-05


#### FastText

In [23]:
# Train the FastText model
model = FastText(sentences=train_tokenized, vector_size=100, window=5, min_count=1, workers=4, sg=0, hs=5, negative=0)

In [24]:
# Save the model
model.save("fasttext_model.model")

# Load the trained model
model = FastText.load("fasttext_model.model")

In [25]:
# Define the prediction function
def predict_next_word(model, context, top_n=5):
    # Preprocess the context
    context = preprocess_text(context)
    words = context.split()

    # Use the last word(s) as the context
    if not words:
        return []

    last_word = words[-1]

    if last_word not in model.wv:
        return []

    # Find the top_n most similar words
    similar_words = model.wv.most_similar(last_word, topn=top_n)

    # Extract the words from the similar words list
    next_words = [word for word, similarity in similar_words]

    return next_words

context = "me pɛ"

In [26]:
# Predict the next word
next_words = predict_next_word(model, context)
print("Next word predictions:", next_words)

# Save word embeddings as text
def save_word_embeddings_as_text(model, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for word in model.wv.index_to_key:
            vector = model.wv[word]
            vector_str = ' '.join(map(str, vector))
            file.write(f"{word} {vector_str}\n")

model.wv.save("fasttext_embeddings.kv")

Next word predictions: ['nnyɛ', 'wɔanhwɛ', 'wɔrekɔhwɛ', 'wɔreyɛ', 'yɛahyɛ']


In [27]:
# Evaluate the model
def evaluate_fasttext_model(model, tokenized_sentences, top_n=5):
    total = 0
    correct = 0

    for sentence in tokenized_sentences:
        for i in range(1, len(sentence)):
            context = ' '.join(sentence[:i])
            true_next_word = sentence[i]
            predicted_next_words = predict_next_word(model, context, top_n=top_n)

            if true_next_word in predicted_next_words:
                correct += 1
            total += 1

    accuracy = correct / total
    return accuracy

# Evaluate on the validation set
accuracy = evaluate_fasttext_model(model, validation_tokenized, top_n=5)
print(f"Validation Accuracy: {accuracy}")


Validation Accuracy: 7.497938067031566e-05


In [28]:
# Optimize the FastText model
def optimize_fasttext(sentences, vector_size=300, window=10, min_count=1, workers=4, sg=1, hs=3, negative=0):
    model = FastText(sentences=sentences, vector_size=vector_size, window=window, min_count=min_count, workers=workers, sg=sg, hs=hs, negative=negative)
    return model

# Train the optimized model
# Assuming `optimize_fasttext` is your function to train the FastText model
optimized_model = optimize_fasttext(train_tokenized)

import pickle

# Save the FastText model to a pickle file
with open('fasttext_model.pkl', 'wb') as file:
    pickle.dump(model, file)



In [29]:
# Define the prediction function for the optimized model
def predict_next_word(model, context, top_n=7):
    # Preprocess the context
    context = preprocess_text(context)
    words = context.split()

    # Use the last word(s) as the context
    if not words:
        return []

    last_word = words[-1]

    if last_word not in model.wv:
        return []

    # Find the top_n most similar words
    similar_words = model.wv.most_similar(last_word, topn=top_n)

    # Extract the words from the similar words list
    next_words = [word for word, similarity in similar_words]

    return next_words

In [30]:
# Predict next words for the test dataset
def predict_words_for_test_dataset(model, tokenized_sentences, top_n=7):
    predictions = []

    for sentence in tokenized_sentences:
        sentence_predictions = []
        for i in range(1, len(sentence)):
            context = ' '.join(sentence[:i])
            next_words = predict_next_word(model, context, top_n=top_n)
            sentence_predictions.append((context, next_words))
        predictions.append(sentence_predictions)

    return predictions

# Get predictions for the test dataset
test_predictions = predict_words_for_test_dataset(optimized_model, test_tokenized, top_n=7)

# Print predictions
for sentence_predictions in test_predictions:
    for context, next_words in sentence_predictions:
        print(f"Context: '{context}' -> Predicted next words: {next_words}")

# Save word embeddings as text
def save_word_embeddings_as_text(model, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for word in model.wv.index_to_key:
            vector = model.wv[word]
            vector_str = ' '.join(map(str, vector))
            file.write(f"{word} {vector_str}\n")

save_word_embeddings_as_text(optimized_model, "optimized_fasttext_embeddings.txt")



Streaming output truncated to the last 5000 lines.
Context: 'ne' -> Predicted next words: ['ahaanu', 'annum', 'mpene', 'yadua', 'naduan', 'aduan', 'baanan']
Context: 'baabi' -> Predicted next words: ['nabatoɔ', 'paa', 'basaa', 'abaawea', 'wadi', 'baano', 'nahomatrofoɔ']
Context: 'amamfrafoɔ' -> Predicted next words: ['wammu', 'ayamhyehyɛ', 'amammɔfoɔ', 'meresrɛ', 'redi', 'asɔresɛm', 'dwumadi']
Context: 'wɔ' -> Predicted next words: ['wɔtwerɛɛ', 'wɔtwaa', 'ɛtaa', 'wɔtwerɛ', 'mmaa', 'mmubu', 'mmɑ']
Context: 'paa' -> Predicted next words: ['akristofoɔ', 'basaa', 'wadi', 'baabi', 'anwea', 'di', 'badwatuo']
Context: 'boa' -> Predicted next words: ['boaboa', 'boaa', 'boaboaa', 'dodo', 'nyabor', 'boaeɛ', 'mugye']
Context: 'no' -> Predicted next words: ['kakraa', 'apaadie', 'asɛmmɔne', 'totoo', 'renhu', 'anwie', 'didi']
Context: 'ma' -> Predicted next words: ['manpaninnana', 'mapa', 'amannɔne', 'manpaninabediakyire', 'manpanyindie', 'mamma', 'manpaninadwa']
Context: 'onhu' -> Predicted next wo

In [31]:
# Evaluate the optimized model on the test dataset
test_accuracy = evaluate_fasttext_model(optimized_model, test_tokenized, top_n=5)
print(f"Test Accuracy: {test_accuracy}")

Test Accuracy: 0.00011546455238241859


In [33]:
# Load the optimized model from the pickle file
with open('fasttext_model.pkl', 'rb') as file:
    optimized_model = pickle.load(file)

#### Generating A Story

In [34]:
def generate_story(model, start_prompt, max_words=100):
    story = start_prompt.split()
    recent_words = []

    for _ in range(max_words):
        context = ' '.join(story[-3:])  # Use the last 3 words as context
        next_word_predictions = predict_next_word(model, context, top_n=10)

        if not next_word_predictions:
            break

        # Choose the most probable word that hasn't been used recently
        for word in next_word_predictions:
            if word not in recent_words:
                next_word = word
                break
        else:
            # If all words have been used recently, use the most probable word
            next_word = next_word_predictions[0]

        story.append(next_word)

        # Keep track of recent words
        recent_words.append(next_word)
        if len(recent_words) > 10:  # Adjust this number as needed
            recent_words.pop(0)

    return ' '.join(story)


# Set the starting prompt
start_prompt = "sɛ"

# Generate the story
generated_story = generate_story(optimized_model, start_prompt, max_words=5)

print("Generated Story:")
print(generated_story)

Generated Story:
sɛ yɛreyɛ ɛreyɛ yɛbɛhwehwɛ ɛhwehwɛ ɔnhwehwɛ


### Creating the StreamLite Application

In [36]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [39]:
import streamlit as st
from gensim.models import FastText
import re


# Define functions for text preprocessing and prediction
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip().lower()

def predict_words_for_test_dataset(model, tokenized_sentences, top_n=7):
    predictions = []

    for sentence in tokenized_sentences:
        sentence_predictions = []
        for i in range(1, len(sentence)):
            context = ' '.join(sentence[:i])
            next_words = predict_next_word(model, context, top_n=top_n)
            sentence_predictions.append((context, next_words))
        predictions.append(sentence_predictions)

    return predictions

# Get predictions for the test dataset
test_predictions = predict_words_for_test_dataset(optimized_model, test_tokenized, top_n=7)

def generate_story(model, start_prompt, max_words=100):
    story = start_prompt.split()
    recent_words = []

    for _ in range(max_words):
        context = ' '.join(story[-3:])  # Use the last 3 words as context
        next_word_predictions = predict_next_word(model, context, top_n=10)

        if not next_word_predictions:
            break

        # Choose the most probable word that hasn't been used recently
        for word in next_word_predictions:
            if word not in recent_words:
                next_word = word
                break
        else:
            # If all words have been used recently, use the most probable word
            next_word = next_word_predictions[0]

        story.append(next_word)

        # Keep track of recent words
        recent_words.append(next_word)
        if len(recent_words) > 10:  # Adjust this number as needed
            recent_words.pop(0)

    return ' '.join(story)



# Streamlit app
st.title("Twi Story Generator")

start_prompt = st.text_input("Enter the starting text:", "sɛ")
max_words = st.slider("Number of words to generate:", min_value=10, max_value=100, value=20)

if st.button("Generate Story"):
    generated_story = generate_story(optimized_model, start_prompt, max_words)
    st.write("Generated Story:")
    st.write(generated_story)

2024-08-05 18:47:20.840 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-08-05 18:47:20.847 Session state does not function when running a script without `streamlit run`


In [41]:
pip install pyngrok

In [43]:
!ngrok authtoken 2kFfrJtOj18Tq0wGuH8UwPncrIx_4znKAFccwN2yghYy9TW3X

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [46]:
import os
from pyngrok import ngrok

# Run the Streamlit app
os.system('streamlit run text_generator_app.py &')

# Install ngrok authtoken (replace with your actual authtoken)
!ngrok authtoken 2kFfrJtOj18Tq0wGuH8UwPncrIx_4znKAFccwN2yghYy9TW3X

# Set up ngrok to tunnel the Streamlit port
public_url = ngrok.connect(8501)
print(f'Streamlit app is live at: {public_url}')


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Streamlit app is live at: NgrokTunnel: "https://9e65-34-91-100-56.ngrok-free.app" -> "http://localhost:8501"
